# Notebook 02 (Light): Long-Term Simulation and Ruin Analysis

**Light version with reduced memory requirements**

This notebook performs Monte Carlo simulations to analyze:
- Ruin probabilities at different time horizons (10, 20, 50, 100 years)
- Impact of insurance deductibles and limits on survival
- Distribution of annualized returns (ROE) over long periods

**Note:** This light version reduces N_SIMULATIONS from 1000 to 100 and MAX_YEARS from 1000 to 100

In [ ]:
# Setup and imports
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from pathlib import Path
import time
import psutil
import gc
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

# Configure matplotlib for WSJ style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['grid.alpha'] = 0.3

# Set random seed
np.random.seed(42)

print(f"Available memory: {psutil.virtual_memory().available / 1e9:.1f} GB")
print("Running LIGHT version with reduced parameters")

## 1. Simulation Parameters (Reduced for Memory)

In [ ]:
# Simulation parameters - REDUCED FOR MEMORY
N_SIMULATIONS = 100  # Reduced from 1000
MAX_YEARS = 100      # Reduced from 1000
TIME_HORIZONS = [10, 20, 50, 100]  # Removed 500, 1000

# Insurance scenarios to test (keeping only 3 key scenarios)
insurance_scenarios = [
    {'name': 'No Insurance', 'deductible': float('inf'), 'limit': 0},
    {'name': 'Medium Coverage', 'deductible': 1_000_000, 'limit': 10_000_000},
    {'name': 'Full Coverage', 'deductible': 100_000, 'limit': 50_000_000},
]

# Claim parameters
claim_params = {
    'frequency': 0.1,  # Expected 1 claim every 10 years
    'severity_mean': 5_000_000,
    'severity_std': 3_000_000,
    'cat_frequency': 0.01,  # 1% chance per year of catastrophic event
    'cat_severity_mean': 25_000_000,
    'cat_severity_std': 10_000_000
}

print(f"Parameters for LIGHT version:")
print(f"  Simulations: {N_SIMULATIONS} (was 1000)")
print(f"  Max years: {MAX_YEARS} (was 1000)")
print(f"  Scenarios: {len(insurance_scenarios)} (was 5)")
print(f"  Total simulation-years: {N_SIMULATIONS * MAX_YEARS * len(insurance_scenarios):,} (was 5M)")
print(f"\nInsurance Scenarios:")
for scenario in insurance_scenarios:
    print(f"  {scenario['name']}: Deductible=${scenario['deductible']:,.0f}, Limit=${scenario['limit']:,.0f}")

## 2. Single Simulation Function

In [ ]:
def run_single_simulation(params: Dict) -> Dict:
    """Run a single simulation with given parameters."""
    
    # Unpack parameters
    sim_id = params['sim_id']
    deductible = params['deductible']
    limit = params['limit']
    seed = params['seed']
    
    # Initialize manufacturer
    config = ManufacturerConfig(
        initial_assets=10_000_000,
        asset_turnover_ratio=1.0,
        base_operating_margin=0.08,
        tax_rate=0.25,
        retention_ratio=1.0
    )
    manufacturer = WidgetManufacturer(config)
    
    # Initialize claim generator
    claim_gen = ManufacturingLossGenerator.create_simple(
        frequency=claim_params['frequency'],
        severity_mean=claim_params['severity_mean'],
        severity_std=claim_params['severity_std'],
        seed=seed
    )
    
    # Generate claims for entire period
    regular_claims = claim_gen.generate_claims(MAX_YEARS)
    cat_claims = claim_gen.generate_catastrophic_claims(
        MAX_YEARS,
        claim_params['cat_frequency'],
        claim_params['cat_severity_mean'],
        claim_params['cat_severity_std']
    )
    
    # Combine and sort claims
    all_claims = regular_claims + cat_claims
    claims_by_year = {}
    for claim in all_claims:
        if claim.year not in claims_by_year:
            claims_by_year[claim.year] = []
        claims_by_year[claim.year].append(claim.amount)
    
    # Track results
    ruin_year = None
    annual_returns = []
    
    # Run simulation
    for year in range(MAX_YEARS):
        # Process claims for this year
        if year in claims_by_year:
            for claim_amount in claims_by_year[year]:
                manufacturer.process_insurance_claim(claim_amount, deductible, limit)
        
        # Take annual step
        metrics = manufacturer.step(
            working_capital_pct=0.2,
            letter_of_credit_rate=0.015,
            growth_rate=0.03
        )
        
        # Record ROE
        annual_returns.append(metrics['roe'])
        
        # Check for ruin
        if manufacturer.is_ruined and ruin_year is None:
            ruin_year = year + 1
            break
    
    # Calculate annualized ROE
    if len(annual_returns) > 0:
        annualized_roe = np.mean(annual_returns)
    else:
        annualized_roe = 0
    
    return {
        'sim_id': sim_id,
        'ruin_year': ruin_year,
        'annualized_roe': annualized_roe,
        'final_equity': manufacturer.equity if not manufacturer.is_ruined else 0,
        'total_claims': sum(claim.amount for claim in all_claims)
    }

## 3. Run Simulations for Each Scenario

In [ ]:
# Store results for all scenarios
scenario_results = {}

# Run simulations for each scenario
for scenario in insurance_scenarios:
    print(f"\nRunning {N_SIMULATIONS} simulations for: {scenario['name']}")
    print("=" * 50)
    
    # Prepare parameters
    sim_params = [
        {
            'sim_id': i,
            'deductible': scenario['deductible'],
            'limit': scenario['limit'],
            'seed': 42 + i  # Different seed for each simulation
        }
        for i in range(N_SIMULATIONS)
    ]
    
    # Track performance
    start_time = time.time()
    start_memory = psutil.Process().memory_info().rss / 1e9
    
    # Run simulations sequentially
    results = []
    for i, params in enumerate(sim_params):
        if i % 20 == 0:
            print(f"  Progress: {i}/{N_SIMULATIONS} simulations completed")
        results.append(run_single_simulation(params))
        
        # Garbage collection every 20 simulations to prevent memory buildup
        if i % 20 == 0:
            gc.collect()
    
    # Calculate metrics
    end_time = time.time()
    end_memory = psutil.Process().memory_info().rss / 1e9
    
    print(f"\n  Time taken: {end_time - start_time:.1f} seconds")
    print(f"  Memory used: {end_memory - start_memory:.1f} GB")
    print(f"  Simulations per second: {N_SIMULATIONS / (end_time - start_time):.1f}")
    
    # Store results
    scenario_results[scenario['name']] = pd.DataFrame(results)
    
    # Clean up memory
    gc.collect()

## 4. Calculate Ruin Probabilities

In [ ]:
# Calculate ruin probabilities
ruin_prob_data = []

for scenario_name, results_df in scenario_results.items():
    for horizon in TIME_HORIZONS:
        # Count simulations that failed by this horizon
        ruined = results_df['ruin_year'].notna() & (results_df['ruin_year'] <= horizon)
        ruin_prob = ruined.sum() / len(results_df)
        
        ruin_prob_data.append({
            'Scenario': scenario_name,
            'Horizon': horizon,
            'Ruin_Probability': ruin_prob * 100
        })

ruin_prob_df = pd.DataFrame(ruin_prob_data)

# Pivot for easier visualization
ruin_pivot = ruin_prob_df.pivot(index='Horizon', columns='Scenario', values='Ruin_Probability')

# Display table
print("\nRuin Probability by Time Horizon (%)")
print("=" * 60)
print(ruin_pivot.to_string(float_format='%.1f'))
print("\nNote: Based on", N_SIMULATIONS, "simulations (light version)")

## 5. Visualize Results

In [ ]:
# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Ruin probability curves
for scenario_name in scenario_results.keys():
    scenario_data = ruin_prob_df[ruin_prob_df['Scenario'] == scenario_name]
    ax1.plot(scenario_data['Horizon'], scenario_data['Ruin_Probability'], 
             marker='o', linewidth=2, label=scenario_name)

ax1.set_xlabel('Time Horizon (Years)')
ax1.set_ylabel('Ruin Probability (%)')
ax1.set_title(f'Probability of Ruin by Time Horizon (n={N_SIMULATIONS})')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='best')

# Plot 2: Bar chart for 100-year horizon
horizon_100 = ruin_prob_df[ruin_prob_df['Horizon'] == 100].sort_values('Ruin_Probability')
colors = ['green' if x < 10 else 'orange' if x < 25 else 'red' 
          for x in horizon_100['Ruin_Probability']]

ax2.barh(range(len(horizon_100)), horizon_100['Ruin_Probability'], color=colors)
ax2.set_yticks(range(len(horizon_100)))
ax2.set_yticklabels(horizon_100['Scenario'])
ax2.set_xlabel('Ruin Probability (%)')
ax2.set_title('100-Year Ruin Probability by Insurance Structure')
ax2.grid(True, alpha=0.3, axis='x')

plt.suptitle('LIGHT VERSION - Reduced Sample Size', fontsize=10, y=1.05)
plt.tight_layout()
plt.show()

## Summary

This **light version** demonstrates the same analysis with reduced parameters:
- **100 simulations** instead of 1,000
- **100 years max** instead of 1,000
- **3 scenarios** instead of 5

Total simulation-years: **30,000** instead of 5,000,000

For full results, run the complete version with adequate memory (19+ GB RAM).